In [0]:
high_test = 54

In [0]:
high_testing = ['part_B_final/test_data_high/images/IMG_50.tiff', 'part_B_final/test_data_mids/images/IMG_89.tiff', 'part_B_final/test_data_high/images/IMG_43.tiff', 'part_B_final/test_data_high/images/IMG_16.tiff', 'part_B_final/test_data_high/images/IMG_7.tiff', 'part_B_final/test_data_high/images/IMG_21.tiff', 'part_B_final/test_data_high/images/IMG_10.tiff', 'part_B_final/test_data_high/images/IMG_12.tiff', 'part_B_final/test_data_mids/images/IMG_70.tiff', 'part_B_final/test_data_high/images/IMG_42.tiff', 'part_B_final/test_data_high/images/IMG_39.tiff', 'part_B_final/test_data_mids/images/IMG_95.tiff', 'part_B_final/test_data_high/images/IMG_9.tiff', 'part_B_final/test_data_high/images/IMG_31.tiff', 'part_B_final/test_data_high/images/IMG_48.tiff', 'part_B_final/test_data_high/images/IMG_40.tiff', 'part_B_final/test_data_mids/images/IMG_101.tiff', 'part_B_final/test_data_high/images/IMG_2.tiff', 'part_B_final/test_data_high/images/IMG_35.tiff', 'part_B_final/test_data_high/images/IMG_52.tiff', 'part_B_final/test_data_high/images/IMG_30.tiff', 'part_B_final/test_data_mids/images/IMG_118.tiff', 'part_B_final/test_data_mids/images/IMG_113.tiff', 'part_B_final/test_data_high/images/IMG_24.tiff', 'part_B_final/test_data_high/images/IMG_23.tiff', 'part_B_final/test_data_high/images/IMG_41.tiff', 'part_B_final/test_data_high/images/IMG_13.tiff', 'part_B_final/test_data_high/images/IMG_32.tiff', 'part_B_final/test_data_high/images/IMG_4.tiff', 'part_B_final/test_data_mids/images/IMG_91.tiff', 'part_B_final/test_data_high/images/IMG_51.tiff', 'part_B_final/test_data_mids/images/IMG_109.tiff', 'part_B_final/test_data_high/images/IMG_18.tiff', 'part_B_final/test_data_high/images/IMG_45.tiff', 'part_B_final/test_data_high/images/IMG_33.tiff', 'part_B_final/test_data_high/images/IMG_49.tiff', 'part_B_final/test_data_high/images/IMG_17.tiff', 'part_B_final/test_data_mids/images/IMG_102.tiff', 'part_B_final/test_data_high/images/IMG_47.tiff', 'part_B_final/test_data_high/images/IMG_1.tiff', 'part_B_final/test_data_high/images/IMG_8.tiff', 'part_B_final/test_data_high/images/IMG_19.tiff', 'part_B_final/test_data_high/images/IMG_15.tiff', 'part_B_final/test_data_high/images/IMG_46.tiff', 'part_B_final/test_data_high/images/IMG_22.tiff', 'part_B_final/test_data_high/images/IMG_14.tiff', 'part_B_final/test_data_high/images/IMG_5.tiff', 'part_B_final/test_data_high/images/IMG_25.tiff', 'part_B_final/test_data_high/images/IMG_44.tiff', 'part_B_final/test_data_high/images/IMG_26.tiff', 'part_B_final/test_data_high/images/IMG_29.tiff', 'part_B_final/test_data_mids/images/IMG_93.tiff', 'part_B_final/test_data_high/images/IMG_27.tiff', 'part_B_final/test_data_mids/images/IMG_97.tiff']

In [39]:
print(len(high_testing))

54


In [0]:
import sys
import os

import warnings




import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

import numpy as np
import argparse
import json
import cv2
import dataset
import time

In [0]:
import torch.nn as nn
import torch
from torch.nn import functional as F
from torchvision import models

class ContextualModule(nn.Module):
    def __init__(self, features, out_features=512, sizes=(1, 2, 3, 6)):
        super(ContextualModule, self).__init__()
        self.scales = []
        self.scales = nn.ModuleList([self._make_scale(features, size) for size in sizes])
        self.bottleneck = nn.Conv2d(features * 2, out_features, kernel_size=1)
        self.relu = nn.ReLU()
        self.weight_net = nn.Conv2d(features,features,kernel_size=1)

    def __make_weight(self,feature,scale_feature):
        weight_feature = feature - scale_feature
        return F.sigmoid(self.weight_net(weight_feature))

    def _make_scale(self, features, size):
        prior = nn.AdaptiveAvgPool2d(output_size=(size, size))
        conv = nn.Conv2d(features, features, kernel_size=1, bias=False)
        return nn.Sequential(prior, conv)

    def forward(self, feats):
        h, w = feats.size(2), feats.size(3)
        multi_scales = [F.upsample(input=stage(feats), size=(h, w), mode='bilinear') for stage in self.scales]
        weights = [self.__make_weight(feats,scale_feature) for scale_feature in multi_scales]
        overall_features = [(multi_scales[0]*weights[0]+multi_scales[1]*weights[1]+multi_scales[2]*weights[2]+multi_scales[3]*weights[3])/(weights[0]+weights[1]+weights[2]+weights[3])]+ [feats]
        bottle = self.bottleneck(torch.cat(overall_features, 1))
        return self.relu(bottle)

class CANNet(nn.Module):
    def __init__(self, load_weights=False):
        super(CANNet, self).__init__()
        self.seen = 0
        self.context = ContextualModule(512, 512)
        self.frontend_feat = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512]
        self.backend_feat  = [512, 512, 512,256,128,64]
        self.frontend = make_layers(self.frontend_feat)
        self.backend = make_layers(self.backend_feat,in_channels = 512,batch_norm=True, dilation = True)
        self.output_layer = nn.Conv2d(64, 1, kernel_size=1)
        if not load_weights:
            mod = models.vgg16(pretrained = True)
            self._initialize_weights()
            for i in xrange(len(self.frontend.state_dict().items())):
                self.frontend.state_dict().items()[i][1].data[:] = mod.state_dict().items()[i][1].data[:]

    def forward(self,x):
        x = self.frontend(x)
        x = self.context(x)
        return x
        x = self.backend(x)
        x = self.output_layer(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

def make_layers(cfg, in_channels = 3,batch_norm=False,dilation = False):
    if dilation:
        d_rate = 2
    else:
        d_rate = 1
    layers = []
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=d_rate,dilation = d_rate)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

In [0]:
model = CANNet()

In [0]:
model = model.cuda()

In [44]:
checkpoint = torch.load('ALL_WEIGHTS/checkpoint_high_separate.tar')

model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:


import h5py
import PIL.Image as Image
import numpy as np
import os
import glob
import scipy
from image import *
from model import CANNet
import torch
from torch.autograd import Variable

from sklearn.metrics import mean_squared_error,mean_absolute_error

from torchvision import transforms

In [0]:
transform=transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

In [0]:
names =[]
for i in range(high_test):
  names.append('IMG_'+str(i+1))

In [0]:
model.eval()


gt = []

In [0]:
for i in xrange(len(high_testing)):
    gt_file = h5py.File(high_testing[i].replace('.tiff','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    gt.append(round(np.sum(groundtruth)))

In [0]:
arrgt = np.asarray(gt)
arrnames = np.asarray(names)

In [53]:
print(arrgt.shape)
print(arrnames.shape)

(54,)
(54,)


In [54]:
print(arrgt)
print(arrnames)

[67. 32. 67. 57. 49. 70. 50. 45. 33. 95. 57. 40. 42. 53. 70. 44. 39. 45.
 52. 79. 46. 35. 36. 41. 43. 47. 55. 47. 44. 37. 63. 39. 50. 98. 47. 61.
 48. 40. 72. 46. 54. 44. 59. 62. 45. 53. 43. 41. 65. 46. 43. 40. 41. 33.]
['IMG_1' 'IMG_2' 'IMG_3' 'IMG_4' 'IMG_5' 'IMG_6' 'IMG_7' 'IMG_8' 'IMG_9'
 'IMG_10' 'IMG_11' 'IMG_12' 'IMG_13' 'IMG_14' 'IMG_15' 'IMG_16' 'IMG_17'
 'IMG_18' 'IMG_19' 'IMG_20' 'IMG_21' 'IMG_22' 'IMG_23' 'IMG_24' 'IMG_25'
 'IMG_26' 'IMG_27' 'IMG_28' 'IMG_29' 'IMG_30' 'IMG_31' 'IMG_32' 'IMG_33'
 'IMG_34' 'IMG_35' 'IMG_36' 'IMG_37' 'IMG_38' 'IMG_39' 'IMG_40' 'IMG_41'
 'IMG_42' 'IMG_43' 'IMG_44' 'IMG_45' 'IMG_46' 'IMG_47' 'IMG_48' 'IMG_49'
 'IMG_50' 'IMG_51' 'IMG_52' 'IMG_53' 'IMG_54']


In [0]:
features_savepath = 'BEST_TEST_HIGH/' 
if not os.path.exists(features_savepath):
    os.makedirs(features_savepath)

In [0]:
np.save(features_savepath +'/ground_truth_counts_test_high.npy', arrgt)
np.save(features_savepath +'/ground_truth_names_test_high.npy', arrnames)

In [0]:

class featureExtractor(nn.Module):
    def __init__(self):
        super(featureExtractor, self).__init__()
        self.context = model.context
        self.frontend = model.frontend

    def forward(self,x):
        x = self.frontend(x)
        x = self.context(x)
        return x



net_conv1 = featureExtractor()


In [0]:
net_conv1 = net_conv1.cuda()

In [0]:
x =[]


In [0]:
for i in xrange(len(high_testing)):
    img = transform(Image.open(high_testing[i]).convert('RGB')).cuda()
    img = img.unsqueeze(0)
    h,w = img.shape[2:4]
    density= net_conv1(img).data.cpu().numpy()
    act = np.squeeze(density)
    pure_name = os.path.splitext(os.path.basename(high_testing[i]))[0]
    x.append(act)
    

    #density = np.squeeze(3)
    



In [61]:
print(len(x))

54


In [0]:
all_feats = []

In [0]:
for i in range(high_test):
  feats = []
  for j in range(512):
    s = 0
    for k in range (42):
      for l in range(42):
        s = s + x[i][j][k][l]
    a = s/1764
    feats.append(a)
  all_feats.append(feats)

In [0]:
f = np.array(all_feats)

In [0]:
print(f.shape)

In [0]:
features = 'BEST_TEST_HIGH/FEATURES/' 
if not os.path.exists(features):
    os.makedirs(features)

In [0]:
for i in range(high_test):
  x=np.array(f[i])
  np.save(features+'/IMG_'+str(i+1)+'.npy',x)